### Direct Get to Drive

In [ ]:
#@title { form-width: "2%" }
#@markdown #### Setup

import os, pytz, time, re
from bs4 import BeautifulSoup as Soup, element
from google.colab import drive
from datetime import datetime


tz = pytz.timezone('Turkey')
os.environ['TZ'] = 'Turkey'
nowstr = lambda separete = False: datetime.now(tz).strftime("%y-%m-%d-%H-%M-%S") if separete else str(round(time.time()))
file_size_limit = 40000 # 40 KB

ddyPtr = re.compile("mydaddy[^']+(?=')")
hqpVarsPtr = re.compile('(?<=;)[a-z]{8}(?==)')
fileExtPtr = re.compile('\.([a-zA-Z0-9]+)$')
hqpTitlePtn = re.compile("(?<=<title>)[\sa-zA-Z0-9]+(?=\s-)")

if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')

def has_size(path, only_check = True):
    size = 0
    if not os.path.exists(path):
        return 0
    if os.path.isfile(path):
        size = os.path.getsize(path)
    else:  
        for path, dirs, files in os.walk(path):
            for file in files:
                filepath = os.path.join(path, file)
                size += os.path.getsize(filepath)
                if only_check and size >= file_size_limit:
                    return True
    if size >= file_size_limit:
        return size
    else:
        return 0
        
def download_file(url, save_dir= "/content/Downloads", name= "", headers= ""):
    tempsave = "/content/done/" + nowstr() 
    os.makedirs(tempsave, exist_ok=True)
    name = name.strip()
    cm = f"wget {headers}" if headers else "wget"
    if os.path.isfile("/content/cf"):
        cm += " --load-cookies /content/cf"
    cm += f' -P "{tempsave}"'

    if name:
        extSch = fileExtPtr.search(url)
        ext = extSch.group() if extSch else ""
        if not name.endswith(ext):
            name = name + ext
        cm += f' -O "{tempsave}/{name}"' 

    cm += f' "{url}"'
    print(f"  {cm}")
    !$cm
    if not has_size(tempsave): 
        print("Download failed!")
        return
    os.makedirs(save_dir, exist_ok=True)
    !mv "$tempsave"/* "$save_dir"

def hqpGetInnerURL(url):
    reqOutput = !curl $url
    outputStr = "".join(reqOutput)
    srch = ddyPtr.search(outputStr)
    if srch:
        titleSH = hqpTitlePtn.search(outputStr)
        title = titleSH.group() if titleSH else ""
        innerURL = f"https://{srch.group()}"
        return (innerURL, title, outputStr)
    else:
        print("Search is ", type(srch))
        return (None, None, outputStr)

def hCCDownload(hURL, saveDir= "/content", name= ""):
    print(f"Looking for cc url in: {hURL}\n\t...\n")
    reqOutput = !curl $hURL
    invalid = False
    if reqOutput:
        if not name:
            name = nowstr(True) + "- 1080p"
        newOutputStr = reqOutput[0]
        varRes = hqpVarsPtr.findall(newOutputStr)
        if varRes:
            valDict = {}
            for mtch in varRes:
                valPt = f'(?<={mtch}=")(\/\/|\d)[^"]+'
                valSch = re.search(valPt, newOutputStr)
                if not valSch:
                    print(f"Value for {mtch} wasn't found!")
                    invalid = True
                    break
                valDict[mtch] = valSch.group()
            if not invalid:
                val1Sch = re.search('(?<=replaceAll\("nrpuv",)[a-z]{8}', newOutputStr)
                val2Sch = re.search('(?<=\+"pubs\/"\+)[a-z]{8}', newOutputStr)
                if val1Sch and val2Sch:
                    val1 = val1Sch.group()
                    val2 = val2Sch.group()
                    vidUrl = f"https:{valDict[val1]}pubs/{valDict[val2]}/1080.mp4"
                else:
                    invalid = True
            if not invalid:
                print(f"Found URL:\n\t{vidUrl}\n\n")
                download_file(vidUrl, save_dir= saveDir, name= name)
            else:
                print(valDict, "\n", f"val1: {val1}, val2: {val2}")


In [ ]:
#@title { form-width: "2%" }
#@markdown


In [ ]:
#@title { form-width: "2%" }
#@markdown #### Direct

Name = "" #@param {type:"string"}
URL = "" #@param {type:"string"}
Ref = "" #@param {type:"string"}
Save_Dir = "" #@param {type:"string"}
PTC = "" #@param {type:"string"}

cookie = """
"""

if len(cookie) > 100:
    with open("cf", "w") as file:
        file.write(cookie)
elif os.path.isfile("/content/cf"):   
    !rm "/content/cf"

headers = ""
if PTC:
    headers = f'--header="accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="accept-encoding: gzip, deflate, br, zstd" --header="accept-language: fr-FR,fr;q=0.9" --header=\'cookie: {PTC}\' --header="dnt: 1" --header="priority: u=0, i" --header="referer: {Ref}" --header=\'sec-ch-ua: "Chromium";v="136", "Google Chrome";v="136", "Not.A/Brand";v="99"\' --header="sec-ch-ua-mobile: ?0" --header=\'sec-ch-ua-platform: "Windows"\' --header="sec-fetch-dest: document" --header="sec-fetch-mode: navigate" --header="sec-fetch-site: same-origin" --header="sec-fetch-user: ?1" --header="upgrade-insecure-requests: 1" --header=\'user-agent: Mozilla/5.0 Windows NT 10.0; Win64; x64 AppleWebKit/537.36 KHTML, like Gecko Chrome/136.0.0.0 Safari/537.36\''

if not Name.endswith(".mp4") and not Name.endswith(".mkv"):
    Name += ".mp4"

if URL:
    download_file(URL, save_dir= Save_Dir, name= Name, headers= headers)

In [ ]:
#@title { form-width: "2%" }
#@markdown #### Find

loc = "" #@param {type:"string"}
srch = "" #@param {type:"string"}
!find "$loc" -name "*$srch*"

In [ ]:
#@title { form-width: "2%" }
#@markdown #### JUST CURL

URL = "" #@param {type:"string"}

!curl $URL

In [ ]:
#@title { form-width: "2%" }
#@markdown #### HQP

VidURLs = "" #@param {type:"string"}
Save_Dir = "" #@param {type:"string"}
Names = "" #@param {type:"string"}

urls = VidURLs.split(",,,")
names = Names.split(",,,")
if len(urls) == len(names):
    print(f"Downloading {len(urls)} links.")
    for i in range(len(urls)):
        url = urls[i].strip()
        name = names[i].strip().title()
        if url and name:
            print(f"\nDownloading {i+1} of {len(urls)}:\t{name}")
            hCCDownload(url, saveDir= Save_Dir, name= name)
        else:
            print("URL or Name is empty!")
else:
    raise("URLs and Names have different length!")

In [ ]:
#@title { form-width: "2%" }
#@markdown
